In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHgD_Fq6TMGseMpTy0yG2R_dfGTU_nYzhcJ7bH1IbheTJxUnDJs2bs
Mounted at /content/drive


In [2]:
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt

import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
type(trainset.targets)
type(trainset.data)

numpy.ndarray

In [5]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

foreground_classes = {'plane', 'car', 'bird'}

background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

fg1,fg2,fg3 = 0,1,2

In [6]:
is_fg = [np.where(np.array(trainset.targets)==fg1)[0] , np.where(np.array(trainset.targets)==fg2)[0], np.where(np.array(trainset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   29    30    35 ... 49987 49991 49995] (15000,)


In [7]:
all_index = np.arange(0,50000)
all_index

array([    0,     1,     2, ..., 49997, 49998, 49999])

In [8]:
train_labels = np.array(trainset.targets)
train_labels.shape

(50000,)

In [9]:
train_labels[is_fg]=1
train_labels[list(set(all_index)-set(is_fg))]=0
train_labels

array([0, 0, 0, ..., 0, 1, 1])

In [10]:
np.count_nonzero(train_labels)

15000

In [11]:
trainset.targets = train_labels

In [12]:
is_fg = [np.where(np.array(testset.targets)==fg1)[0] , np.where(np.array(testset.targets)==fg2)[0], np.where(np.array(testset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   3   10   21 ... 9970 9982 9989] (3000,)


In [13]:
all_index = np.arange(0,10000)
all_index

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [14]:
test_labels = np.array(testset.targets)
test_labels.shape

(10000,)

In [15]:
test_labels[is_fg]=1
test_labels[list(set(all_index)-set(is_fg))]=0
test_labels

array([0, 0, 0, ..., 0, 1, 0])

In [16]:
np.count_nonzero(test_labels)

3000

In [17]:
testset.targets = test_labels

In [18]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=False)

In [63]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=4, padding=0)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(in_channels=5, out_channels=6, kernel_size=4, padding=0)
    # self.conv3 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, padding=0)
    self.fc1 = nn.Linear(726, 256)
    self.fc2 = nn.Linear(256, 64)
    self.fc3 = nn.Linear(64, 10)
    self.fc4 = nn.Linear(10,2)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    # x = self.pool(F.relu(self.conv2(x)))
    # print(x.shape)
    x = (F.relu(self.conv2(x)))
    x =  x.view(x.size(0), -1)
    # print(x.shape)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [64]:
where_net = CNN()#.double()
where_net = where_net.to("cuda")

In [65]:
where_net

CNN(
  (conv1): Conv2d(3, 5, kernel_size=(4, 4), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(5, 6, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=726, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
)

In [66]:
import torch.optim as optim
criterion_where = nn.CrossEntropyLoss()
optimizer_where = optim.SGD(where_net.parameters(), lr=0.01, momentum=0.9)

In [67]:
acti = []
loss_curi = []
epochs = 1000
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_where.zero_grad()

        # forward + backward + optimize
        outputs = where_net(inputs)
        loss = criterion_where(outputs, labels)
        loss.backward()
        optimizer_where.step()

        # print statistics
        running_loss += loss.item()
        mini_batch = 50
        if i % mini_batch == mini_batch-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / mini_batch))
            ep_lossi.append(running_loss/mini_batch) # loss per minibatch
            running_loss = 0.0
            
    if(np.mean(ep_lossi) <= 0.005):
      break;
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch            

print('Finished Training')

[1,    50] loss: 0.621
[1,   100] loss: 0.607
[1,   150] loss: 0.614
[2,    50] loss: 0.615
[2,   100] loss: 0.607
[2,   150] loss: 0.610
[3,    50] loss: 0.609
[3,   100] loss: 0.611
[3,   150] loss: 0.610
[4,    50] loss: 0.606
[4,   100] loss: 0.617
[4,   150] loss: 0.612
[5,    50] loss: 0.604
[5,   100] loss: 0.607
[5,   150] loss: 0.605
[6,    50] loss: 0.584
[6,   100] loss: 0.581
[6,   150] loss: 0.565
[7,    50] loss: 0.541
[7,   100] loss: 0.535
[7,   150] loss: 0.528
[8,    50] loss: 0.507
[8,   100] loss: 0.501
[8,   150] loss: 0.496
[9,    50] loss: 0.495
[9,   100] loss: 0.474
[9,   150] loss: 0.477
[10,    50] loss: 0.480
[10,   100] loss: 0.460
[10,   150] loss: 0.457
[11,    50] loss: 0.449
[11,   100] loss: 0.449
[11,   150] loss: 0.442
[12,    50] loss: 0.437
[12,   100] loss: 0.435
[12,   150] loss: 0.438
[13,    50] loss: 0.425
[13,   100] loss: 0.417
[13,   150] loss: 0.437
[14,    50] loss: 0.417
[14,   100] loss: 0.414
[14,   150] loss: 0.418
[15,    50] loss: 0

In [68]:
torch.save(where_net.state_dict(),"/content/drive/My Drive/Research/Cheating_data/Focus_net_weights/focus_net_2layer_cnn_5_6.pt")

In [69]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = where_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 50000 train images: 99 %
50000 49942


In [70]:
correct = 0
total = 0
out = []
pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= where_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 80 %
10000 8090
